Let's play around with the graphviz backend.

### Shift register

This example is taken from the shift_register notebook.

In [1]:
from magma import *
from magma.primitives import DefineRegister

N = 4
Register4 = DefineRegister(4)
T = Bits(N)

class ShiftRegister(Circuit):
    name = "ShiftRegister"
    IO = ["I", In(T), "O", Out(T), "CLK", In(Bit)]
    @classmethod
    def definition(io):
        regs = [Register4() for _ in range(N)]
        [wire(io.CLK, reg.clk) for reg in regs]  # TODO: Clean up this clock wiring
        wire(io.I, getattr(regs[0], "in"))
        braid(regs, foldargs={"in":"out"})
        wire(regs[-1].out, io.O)


Now let's visualize our `ShiftRegister` type.

In [2]:
ShiftRegister

ShiftRegister = DefineCircuit("ShiftRegister", "I", In(Bits(4)), "O", Out(Bits(4)), "CLK", In(Bit))
inst0 = coreir_reg_P(width="4")
inst1 = coreir_reg_P(width="4")
inst2 = coreir_reg_P(width="4")
inst3 = coreir_reg_P(width="4")
EndCircuit()

For some separate reason, the visualization is slightly broken (outputs aren't wired up) for a placed circuit / instance (?) like `ShiftRegister()` below.

It's not graphviz-specific, I think: the same issue shows up in firrtl and verilog compile output.

In [3]:
ShiftRegister()

= ShiftRegister()

### Simple ALU

Another example, taken from the Chisel examples folder.

In [4]:
from functools import reduce

def one_hot_mux(conds, inputs):
    outputs = []
    for cond, inp in zip(conds, inputs):
        outputs.append(inp & uint([cond for _ in range(len(inp))]))
    return reduce(lambda x, y: x | y, outputs)


class SimpleALU(Circuit):
    name = "SimpleALU"
    IO = ["a", In(UInt(4)), "b", In(UInt(4)), "opcode", In(UInt(2)), "out", Out(UInt(4))]
    
    @classmethod
    def definition(io):
        is_op0 = io.opcode == uint(0, n=2)
        is_op1 = io.opcode == uint(1, n=2)
        is_op2 = io.opcode == uint(2, n=2)
        is_op3 = io.opcode == uint(3, n=2)
        op0_out = io.a + io.b
        op1_out = io.a - io.b
        op2_out = io.a
        op3_out = io.b
        wire(io.out, one_hot_mux([is_op0, is_op1, is_op2, is_op3], [op0_out, op1_out, op2_out, op3_out]))

In [5]:
SimpleALU

SimpleALU = DefineCircuit("SimpleALU", "a", In(UInt(4)), "b", In(UInt(4)), "opcode", In(UInt(2)), "out", Out(UInt(4)))
inst0 = coreir_eq2(width="2")
inst1 = coreir_eq2(width="2")
inst2 = coreir_eq2(width="2")
inst3 = coreir_eq2(width="2")
inst4 = coreir_add4(width="4")
inst5 = coreir_sub4(width="4")
inst6 = coreir_and4(width="4")
inst7 = coreir_and4(width="4")
inst8 = coreir_and4(width="4")
inst9 = coreir_and4(width="4")
inst10 = coreir_or4(width="4")
inst11 = coreir_or4(width="4")
inst12 = coreir_or4(width="4")
wire(0, inst0.in1[0])
wire(0, inst0.in1[1])
wire(1, inst1.in1[0])
wire(0, inst1.in1[1])
wire(0, inst2.in1[0])
wire(1, inst2.in1[1])
wire(1, inst3.in1[0])
wire(1, inst3.in1[1])
wire(inst0.out, inst6.in1[0])
wire(inst0.out, inst6.in1[1])
wire(inst0.out, inst6.in1[2])
wire(inst0.out, inst6.in1[3])
wire(inst1.out, inst7.in1[0])
wire(inst1.out, inst7.in1[1])
wire(inst1.out, inst7.in1[2])
wire(inst1.out, inst7.in1[3])
wire(inst2.out, inst8.in1[0])
wire(inst2.out, inst8.in1[1])
wire(inst2.out, inst8.in1[2])
wire(inst2.out, inst8.in1[3])
wire(inst3.out, inst9.in1[0])
wire(inst3.out, inst9.in1[1])
wire(inst3.out, inst9.in1[2])
wire(inst3.out, inst9.in1[3])
EndCircuit()